In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datos sintéticos: función no lineal
def generate_data(n_samples=1000):
    x = torch.linspace(-2, 2, n_samples).unsqueeze(1).to(device)
    y = torch.sin(3 * x) + 0.3 * torch.cos(5 * x)
    return x, y

x_train, y_train = generate_data()

# Funciones de activación disponibles
activation_dict = {
    "ReLU": nn.ReLU,
    "LeakyReLU": lambda: nn.LeakyReLU(0.01),
    "ELU": nn.ELU,
    "Sigmoid": nn.Sigmoid,
    "Tanh": nn.Tanh
}

# Red neuronal configurable
class SimpleNet(nn.Module):
    def __init__(self, hidden_size=64, activation=nn.ReLU):
        super().__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
        self.activation = activation()

    def forward(self, x):
        a1 = self.activation(self.fc1(x))
        a2 = self.activation(self.fc2(a1))
        a3 = self.activation(self.fc3(a2))
        out = self.fc4(a3)
        return out, [a1, a2, a3]

# Función de entrenamiento + visualización
def train_and_plot(activation_name, hidden_size):
    model = SimpleNet(hidden_size=hidden_size, activation=activation_dict[activation_name]).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()

    losses = []
    epochs = 200
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs, _ = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    # Evaluar modelo y obtener activaciones
    model.eval()
    with torch.no_grad():
        outputs, activations = model(x_train)

    # Calcular neuronas muertas
    dead_neurons = [(act == 0).sum().item() for act in activations]
    total_neurons = [act.numel() for act in activations]

    # Graficar activaciones
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    for i, act in enumerate(activations, 1):
        plt.plot(act[:100].cpu().numpy(), label=f'Capa {i}')
    plt.title(f"Activaciones por capa - {activation_name}, {hidden_size} neuronas")
    plt.xlabel("Ejemplo")
    plt.ylabel("Valor activación")
    plt.legend()

    # Graficar pérdida
    plt.subplot(1, 2, 2)
    plt.plot(losses)
    plt.title("Velocidad de convergencia (MSE por época)")
    plt.xlabel("Época")
    plt.ylabel("MSE")

    plt.subplot(1, 2, 3)
    # Histograma de activaciones
    plt.hist(out.detach().cpu().numpy().flatten(), bins=50, color='skyblue')
    plt.title(f'Distribución de activaciones con {activation_name}')
    plt.xlabel('Activación')
    plt.ylabel('Frecuencia')
    plt.grid(True)
    plt.show()

    plt.tight_layout()
    plt.show()

    # Mostrar neuronas muertas
    print("💀 Neuronas muertas por capa oculta:")
    for i, (dead, total) in enumerate(zip(dead_neurons, total_neurons), 1):
        pct = 100 * dead / total
        print(f"  Capa {i}: {dead}/{total} ({pct:.2f}%) muertas")

# Widget interactivo
interact(
    train_and_plot,
    activation_name=widgets.Dropdown(options=list(activation_dict.keys()), value='ReLU', description='Activación'),
    hidden_size=widgets.IntSlider(value=64, min=8, max=256, step=8, description='Tamaño capa')
)

ModuleNotFoundError: No module named 'tensorflow'